In [68]:
import pandas as pd


train_df = pd.read_json('new_data.json')

train_df["num_photos"] = train_df["photos"].apply(len)
train_df["num_features"] = train_df["features"].apply(len)
train_df["num_description"] = train_df["description"].apply(len)
train_df["created"] = pd.to_datetime(train_df["created"])
train_df["created_year"] = train_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
# train_df.head(100)

In [69]:
column = train_df.columns
column

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude', 'listing_id', 'longitude',
       'manager_id', 'photos', 'price', 'street_address', 'interest_level',
       'num_photos', 'num_features', 'num_description', 'created_year',
       'created_month', 'created_day'],
      dtype='object')

In [70]:
latitudeMean = train_df['latitude'].mean()
longitudeMean = train_df['longitude'].mean()

In [71]:
from geopy import distance

In [72]:
site_coords=(latitudeMean, longitudeMean)
site_coords

(40.75186182911392, -73.97414270675107)

In [73]:
train_df['distance'] = train_df.apply(lambda row:distance.distance(site_coords, (row.latitude, row.longitude)).km, axis=1)

In [74]:
num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description",
             "created_year", "created_month", "created_day"]
num_feats = ["bathrooms", "bedrooms",'distance','price',"num_photos", "num_features", "num_description",
             "created_year", "created_month", "created_day"]
#num_feats = ['bathrooms', "bedrooms",'distance','price']
Y = pd.DataFrame()
X = train_df[num_feats]
Y['interest_level'] = train_df['interest_level']
# X.head(2)

In [75]:
X.to_csv("KYdata/test_x.csv", index=False)
Y.to_csv("KYdata/test_y.csv", index=False)


In [76]:
from sklearn import svm
from sklearn.model_selection import train_test_split

from sklearn import metrics

X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.33)

In [77]:
clf = svm.SVC(kernel='sigmoid', C=1,probability=True).fit(X_train, y_train.values.ravel())
print(clf.score(X_test, y_test))
print(clf.score(X_train, y_train))

0.5404040404040404
0.5362113483216827


In [78]:
# from sklearn.model_selection import cross_val_score

# classifier = svm.SVC(kernel='rbf', C=1,probability=True)
# scores = cross_val_score(classifier, X, Y, cv=5)
# scores

In [79]:
# print("Accuracy : %0.3f (+/- %0.4f )" % (scores.mean(), scores.std()*2))

In [80]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits = 3)
# kf

In [81]:
df = pd.read_json(open("test.json", "r"))
print(df.shape)
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day

(74659, 14)


In [82]:
latitudeMean = df['latitude'].mean()
print(latitudeMean)
longitudeMean = df['longitude'].mean()
print(longitudeMean)
site_coords=(latitudeMean, longitudeMean)
df['distance'] =df.apply(lambda row:distance.distance(site_coords, (row.latitude, row.longitude)).km, axis=1)

40.73505991775942
-73.94528190974967


In [83]:
column = df.columns
column

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude', 'listing_id', 'longitude',
       'manager_id', 'photos', 'price', 'street_address', 'num_photos',
       'num_features', 'num_description', 'created_year', 'created_month',
       'created_day', 'distance'],
      dtype='object')

In [84]:
# num_feats = ["bathrooms", "bedrooms","latitude", "longitude",'distance','price','num_photos', 'num_features','num_description','created_month','created_day']
test_listingID = pd.DataFrame()
test_x=df[num_feats] 
test_listingID['listing_id'] = df['listing_id']
test_x.to_csv("KYdata/traget_x.csv", index=False)
test_listingID.to_csv("KYdata/target_listing.csv", index=False)


In [85]:
test_x
test_listingID

,listing_id
0,7142618
1,7210040
2,7174566
3,7191391
5,7171695
...,...
124003,6928108
124005,6906674
124006,6897967
124007,6842183


In [86]:
y = clf.predict_proba(test_x)
print("Done")

Done


In [87]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [88]:
final_test = pd.DataFrame(y, columns=['high','low','medium'])
final_test = final_test[['high', 'medium','low']]
final_test

,high,medium,low
0,0.017853,0.214116,0.768031
1,0.017587,0.213779,0.768634
2,0.019050,0.213500,0.767449
3,0.017702,0.213938,0.768360
4,0.019502,0.215512,0.764986
...,...,...,...
74654,0.029317,0.213724,0.756959
74655,0.032020,0.218404,0.749576
74656,0.018329,0.213348,0.768324
74657,0.636312,0.129489,0.234199


In [89]:
test_listingID.reset_index(drop=True, inplace=True)
final_test.reset_index(drop=True, inplace=True)
final_test = pd.concat( [test_listingID, final_test], axis=1) 
final_test

,listing_id,high,medium,low
0,7142618,0.017853,0.214116,0.768031
1,7210040,0.017587,0.213779,0.768634
2,7174566,0.019050,0.213500,0.767449
3,7191391,0.017702,0.213938,0.768360
4,7171695,0.019502,0.215512,0.764986
...,...,...,...,...
74654,6928108,0.029317,0.213724,0.756959
74655,6906674,0.032020,0.218404,0.749576
74656,6897967,0.018329,0.213348,0.768324
74657,6842183,0.636312,0.129489,0.234199


In [90]:
final_test.to_csv("output/submission_OOOOKKKK.csv", index=False)